# Perform variant quality control

In this notebook, we perform variant quality control using [PLINK2](https://www.cog-genomics.org/plink/2.0/).

Note that this work is part of a larger project to [Demonstrate the Potential for Pooled Analysis of All of Us and UK Biobank Genomic Data](https://docs.google.com/document/d/19ZS0z_-7FEM37pNDAXaWaqBSLnqyd9MZEkiOmtF3n_0/edit#). Specifically this is for the portion of the project that is the **siloed** analysis.

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the <i>All of Us</i> Workbench.
    <ul>
        <li>Use "Recommended Environment" <kbd><b>General Analysis</b></kbd> which creates compute type <kbd><b>Standard VM</b></kbd> with reasonable defaults for CPU, RAM and disk.</li>
        <li>This notebook can take a while to run <b>TBD DETAILS CHR21 VS. ALL CHRS</b>. Recommend that it is run in the background via <kbd>run_notebook_in_the_background</kbd>.</li>    </ul>
</div>

In [ ]:
from datetime import datetime
import os
import time

## Setup plink2

https://www.cog-genomics.org/plink/2.0/

In [ ]:
%%bash

##### plink 2 install
PLINK_VERSION=2.3.Alpha
PLINK_ZIP_PATH=/tmp/plink-$PLINK_VERSION.zip
curl -L -o $PLINK_ZIP_PATH https://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
mkdir -p /tmp/plink2/
unzip -o $PLINK_ZIP_PATH -d /tmp/plink2/

In [ ]:
!/tmp/plink2/plink2 --version # --help

## Define constants

In [ ]:
# Papermill parameters. See https://papermill.readthedocs.io/en/latest/usage-parameterize.html

#---[ Inputs ]---
# The BGEN file was created via aou_workbench_siloed_analyses/02_aou_write_filtered_bgen.ipynb.
REMOTE_BGEN = 'gs://fc-secure-471c1068-cd3d-4b43-9b5d-a618c85ceea5/data/aou/geno/20220207/aou-alpha3-chr21.bgen'
# The sample file was created via aou_workbench_siloed_analyses/02_aou_write_filtered_bgen.ipynb.
REMOTE_BGEN_SAMPLE = 'gs://fc-secure-471c1068-cd3d-4b43-9b5d-a618c85ceea5/data/aou/geno/20220207/aou-alpha3-chr21.sample'
# This TSV was created via notebook aou_workbench_siloed_analyses/01_aou_lipids_phenotype.ipynb
REMOTE_PHENOTYPE_SAMPLE_IDS = 'gs://fc-secure-471c1068-cd3d-4b43-9b5d-a618c85ceea5/data/aou/pheno/20220208/aou_alpha3_lipids_ids.tsv'

#---[ Outputs ]---
# Create a timestamp for a folder of results generated today.
DATESTAMP = time.strftime('%Y%m%d')
OUTPUT_FILENAME_PREFIX = 'aou_alpha3_lipids'
OUTPUT_FOLDER = f'{os.getenv("WORKSPACE_BUCKET")}/data/aou/variant-qc/{DATESTAMP}/'

In [ ]:
LOCAL_BGEN = os.path.basename(REMOTE_BGEN)
LOCAL_BGEN_SAMPLE = os.path.basename(REMOTE_BGEN_SAMPLE)
LOCAL_PHENOTYPE_SAMPLE_IDS = os.path.basename(REMOTE_PHENOTYPE_SAMPLE_IDS)

## Copy data locally

In [ ]:
!gsutil cp -n {REMOTE_BGEN} {REMOTE_BGEN_SAMPLE} .

In [ ]:
!gsutil cp {REMOTE_PHENOTYPE_SAMPLE_IDS} .

# Variant QC for regenie step 1 via plink2

Use plink2 to perform [variant QC appropriate for step 1 of regenie](https://rgcgithub.github.io/regenie/recommendations/#exclusion-files), obtaining a subset of high quality variants roughly equal to the number of samples.

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_BGEN} ref-first \
  --sample {LOCAL_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_PHENOTYPE_SAMPLE_IDS} \
  --geno 0.1 \
  --mac 100 \
  --hwe 1e-15 \
  --write-snplist \
  --write-samples \
  --no-id-header \
  --out {OUTPUT_FILENAME_PREFIX}_step1QC_plink

# TODO(margaret): add this parameter back in when we have data for all autosomes.
#   --mind 0.1 \

In [ ]:
!ls -lth . | head

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step1QC_plink.id

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step1QC_plink.id

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step1QC_plink.snplist

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step1QC_plink.snplist

In [ ]:
!ls {OUTPUT_FILENAME_PREFIX}_step1QC_plink*

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}_step1QC_plink* {OUTPUT_FOLDER}

# Variant QC for regenie step 2 via plink2

Use plink2 to perform [variant QC appropriate for step 2 of regenie](https://rgcgithub.github.io/regenie/recommendations/#exclusion-files), obtaining a subset of high quality variants.

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_BGEN} ref-first \
  --sample {LOCAL_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_PHENOTYPE_SAMPLE_IDS} \
  --geno 0.1 \
  --hwe 1e-15 \
  --write-snplist \
  --write-samples \
  --no-id-header \
  --out {OUTPUT_FILENAME_PREFIX}_step2QC_plink

# TODO(margaret): add this parameter back in when we have data for all autosomes.
#  --mind 0.1 \


# This differs from step 1 QC in that the following parameter was removed
#    --mac \

In [ ]:
!ls -lth . | head

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step2QC_plink.id

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step2QC_plink.id

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step2QC_plink.snplist

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step2QC_plink.snplist

In [ ]:
!ls {OUTPUT_FILENAME_PREFIX}_step2QC_plink*

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}_step2QC_plink* {OUTPUT_FOLDER}

In [ ]:
!gsutil ls {OUTPUT_FOLDER}

# Provenance 

In [ ]:
%%bash

date

In [ ]:
%%bash

pip3 freeze